In [2]:
import numpy as np
import torch
from glob import glob
import os
import matplotlib.pyplot as plt                        
%matplotlib inline 
os.environ['TORCH_HOME'] = '/content/drive/My Drive/DL_Class/medical_proj/models/'
use_cuda = torch.cuda.is_available()

In [3]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim


model_transfer = models.resnet101(pretrained=True)
if use_cuda:
    model_transfer = model_transfer.cuda()

In [4]:
criterion_transfer = nn.CrossEntropyLoss()
optimizer_transfer = optim.Adam(model_transfer.fc.parameters(), lr=0.001)

In [5]:
from PIL import Image, ImageFile 
from torchvision import datasets
import torchvision.transforms as transforms
import torch.nn as nn


import matplotlib.pyplot as plt                        
%matplotlib inline

ImageFile.LOAD_TRUNCATED_IMAGES = True

# check if CUDA is available
use_cuda = torch.cuda.is_available()

### TODO: Write data loaders for training, validation, and test sets
## Specify appropriate transforms, and batch_sizes
batch_size, num_workers = 20, 2

data_parts = ['train', 'valid', 'test']

transform = transforms.Compose([transforms.Resize(size=224),
                                transforms.CenterCrop((224,224)),
                                transforms.RandomHorizontalFlip(p=.25), # randomly flip and rotate
                                transforms.RandomRotation(10),
                                transforms.RandomVerticalFlip(p=.25),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
data_parts_dir = '/content/drive/My Drive/DL_Class/medical_proj/data'

loaders_scratch = {data_part: torch.utils.data.DataLoader(datasets.ImageFolder(os.path.join(data_parts_dir, data_part), 
                    transform),shuffle=True, batch_size=batch_size, num_workers=num_workers) 
                    for data_part in data_parts}

In [6]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for data, target in loaders['train']:
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass
            loss.backward()
            #parameter update
            optimizer.step()
            # update training loss
            train_loss += loss.item()*data.size(0)
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for data, target in loaders['valid']:
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            # forward pass
            output = model(data)
            #batch loss
            loss = criterion(output, target)
            #update average validation loss 
            valid_loss += loss.item()*data.size(0)
            
        # calculate average losses
        train_loss = train_loss/len(loaders['train'].dataset)
        valid_loss = valid_loss/len(loaders['valid'].dataset)
        
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## TODO: save the model if validation loss has decreased
             # save model if validation loss has decreased
        if (valid_loss <= valid_loss_min):
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
    # return trained model
    return model

In [8]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Testing Loss Average: {:.6f} '.format(test_loss))    
    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))


In [9]:
loaders_transfer = loaders_scratch
print(loaders_transfer)

{'train': <torch.utils.data.dataloader.DataLoader object at 0x7f56162bd2b0>, 'valid': <torch.utils.data.dataloader.DataLoader object at 0x7f56162bd400>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x7f56162bd550>}


In [12]:
model_transfer = train(15, loaders_transfer, model_transfer, optimizer_transfer, criterion_transfer, use_cuda, '/content/drive/My Drive/DL_Class/medical_proj/models/model_transfer.pt')
model_transfer.load_state_dict(torch.load('/content/drive/My Drive/DL_Class/medical_proj/models/model_transfer.pt'))

KeyboardInterrupt: ignored

In [ ]:
model_transfer.load_state_dict(torch.load('/content/drive/My Drive/DL_Class/medical_proj/models/model_transfer.pt'))
test(loaders_transfer, model_transfer, criterion_transfer, use_cuda)